<a href="https://colab.research.google.com/github/TeckniX/dagleash/blob/main/explore_dags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# uncomment to setup GDrive for everything
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# uncomment to clone the repo
#!git clone https://github.com/TeckniX/dagleash.git

fatal: destination path 'dagleash' already exists and is not an empty directory.


In [7]:
# change context to the cloned repo
%cd /content/dagleash

/content/dagleash


In [6]:
DAGSHUB_USER= "lokitek" #@param {type:"string"}
DAGSHUB_REPO_OWNER = "lokitek" #@param {type:"string"}
DAGSHUB_REPO="COCO_1K" #@param {type:"string"}
DAGSHUB_FULL_REPO=DAGSHUB_REPO_OWNER + "/" + DAGSHUB_REPO
DATASOURCE_NAME = "COCO_1K" #@param {type:"string"}
MLFLOW_PROJECT = "Default" #@param {type:"string"}

In [15]:
# Generate random hash for datasource
import hashlib
import secrets

def generate_random_hash(length=32):
    """Generates a random hash of specified length."""

    # Generate random bytes
    random_bytes = secrets.token_bytes(length)

    # Create a hash object (using SHA-256)
    hash_object = hashlib.sha256()

    # Update the hash object with the random bytes
    hash_object.update(random_bytes)

    # Get the hexadecimal representation of the hash
    hex_digest = hash_object.hexdigest()

    return hex_digest

In [8]:
# setup imports
!pip install --upgrade --quiet dagshub torch mlflow ultralytics label_studio_converter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/

In [16]:
# uncomment to download the latest data
#!dagshub download Dean/COCO_1K data --keep-prefix

In [10]:
import yaml
import torch

from utils.config import Config
from utils.dagshub_yolo_cb import custom_callbacks_fn
from utils.data import DataFunctions

import dagshub
from dagshub.data_engine import datasources, datasets

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
classes = Config.classes
dataset_func = DataFunctions(dataset_dir="data/", classes=classes, label_type='segmentation')

#DATASOURCE_NAME = DATASOURCE_NAME + "_" + generate_random_hash()

ds = datasources.create_from_repo(DAGSHUB_FULL_REPO, DATASOURCE_NAME, "data/images")
# ds = datasources.get_datasource(DAGSHUB_FULL_REPO, DATASOURCE_NAME)
# ds.all().download_binary_columns().dataframe
# print(ds.head().dataframe)

Output()

Empty DataFrame
Columns: []
Index: []


In [ ]:
md_query = ds.all()
md = md_query.dataframe
print(md)

enriched_md = md.apply(lambda x: dataset_func.create_metadata(x), axis=1)
print(enriched_md)

Output()

                        path  datapoint_id  \
0     train/000000000009.jpg      81615435   
1     train/000000000025.jpg      81615436   
2     train/000000000030.jpg      81615437   
3     train/000000000034.jpg      81615438   
4     train/000000000036.jpg      81615439   
...                      ...           ...   
1194    val/000000109827.jpg      81616629   
1195    val/000000110042.jpg      81616630   
1196    val/000000112997.jpg      81616631   
1197    val/000000121673.jpg      81616632   
1198    val/000000123633.jpg      81616633   

                                   dagshub_download_url  \
0     https://dagshub.com/api/v1/repos/lokitek/COC...   
1     https://dagshub.com/api/v1/repos/lokitek/COC...   
2     https://dagshub.com/api/v1/repos/lokitek/COC...   
3     https://dagshub.com/api/v1/repos/lokitek/COC...   
4     https://dagshub.com/api/v1/repos/lokitek/COC...   
...                                                 ...   
1194  https://dagshub.com/api/v1/repos/lokit

AttributeError: 'str' object has no attribute 'decode'